In [4]:

#from PyPDF2 import PdfReader
from langchain_community.document_loaders import PyPDFLoader
#from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
#from langchain.vectorstores import FAISS
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate

In [5]:
import os
load_dotenv()
os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

In [6]:
pdfloader = PyPDFLoader("Canada.pdf")
pdfpages = pdfloader.load_and_split()

In [7]:
#Load the document using the PDF loader
mydocuments = pdfloader.load()

In [8]:
# We need to split the text using Character Text Split such that it sshould not increse token size
text_splitter = CharacterTextSplitter(
    # separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 100,
    # length_function = len,
)
texts = text_splitter.split_documents(mydocuments)

In [9]:
# Download embeddings from Gemini Pro
embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")

In [10]:
db = FAISS.from_documents(texts, embeddings)

In [11]:
db

In [12]:
#using langchain retriever
retriever  = db.as_retriever()
from langchain.agents.agent_toolkits import create_retriever_tool
tool = create_retriever_tool(retriever,
    "Canada",
    "Searches and return answer in 2 lines of text"
)
tools = [tool]

In [13]:
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent
llm = ChatGoogleGenerativeAI(model="gemini-pro",temperature=0.3)
agent_executor = create_conversational_retrieval_agent(llm, tools, verbose=True)

In [14]:
Input = "why canada is very cold"
result = db.similarity_search(Input)
result = agent_executor.invoke({"input" : Input})



> Entering new AgentExecutor chain...

Invoking: `Canada` with `{'query': 'why canada is very cold'}`


A topographic map of Canada, in polar projection
(for 90° W), showing elevations shaded from
green to brown (higher)
Köppen climate classification types of Canadato the southeast.[135] Canada is also home to the world's
northernmost settlement, Canadian Forces Station Alert,
on the northern tip of Ellesmere Island—latitude 82.5° N
—which lies 817 kilometres (508 mi) from the North
Pole.[136]
Canada can be divided into seven phys iographic
regions: the Canadian Shield, the interior plains, the
Great Lakes-St. Lawrence Lowlands, the Appalachian
region, the Western Cordillera, Hudson Bay Lowlands,
and the Arctic Archipelago.[137] Boreal forests prevail
throughout  the country, ice is prominent in northern
Arctic regions and through the Rocky Mountains, and
the relatively flat Canadian Prairies in the southwest
facilitate produc tive agriculture.[131] The Great Lakes
feed the St. Lawre